<a href="https://colab.research.google.com/github/KomalSingh11/Fake-News-Prediction/blob/main/Fake_News_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

About the Dataset:

1. id: unique id for a news article
2. title: the title of a news article
3. author: author of the news article
4. text: the text of the article; could be incomplete
5. label: a label that marks whether the news article is real or fake:
           1: Fake news
           0: real News





Importing the libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

Importing the dataset

In [ ]:
news_dataset = pd.read_csv('/content/train.csv')

In [ ]:
news_dataset.shape

(20800, 5)

In [ ]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
news_dataset.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [ ]:
news_dataset = news_dataset.fillna('')

In [ ]:
news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']

In [ ]:
X = news_dataset.drop(columns='label', axis=1)
y = news_dataset['label']

In [ ]:
print(X)
print(y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

Stemming:

Stemming is the process of reducing a word to its Root word

example: actor, actress, acting --> act

In [ ]:
ps = PorterStemmer()

In [ ]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [ps.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return ' '.join([ps.stem(word) for word in content.split()])

In [ ]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [ ]:
print(news_dataset['content'])

0        darrel lucu hous dem aide: we didn’t even see ...
1        daniel j. flynn flynn: hillari clinton, big wo...
2        consortiumnews.com whi the truth might get you...
3        jessica purkiss 15 civilian kill in singl us a...
4        howard portnoy iranian woman jail for fiction ...
                               ...                        
20795    jerom hudson rapper t.i.: trump a ’poster chil...
20796    benjamin hoffman n.f.l. playoffs: schedule, ma...
20797    michael j. de la merc and rachel abram macy’ i...
20798    alex ansari nato, russia to hold parallel exer...
20799                david swanson what keep the f-35 aliv
Name: content, Length: 20800, dtype: object


In [ ]:
X = news_dataset['content'].values
y = news_dataset['label'].values

In [ ]:
print(X)

['darrel lucu hous dem aide: we didn’t even see comey’ letter until jason chaffetz tweet it'
 'daniel j. flynn flynn: hillari clinton, big woman on campu - breitbart'
 'consortiumnews.com whi the truth might get you fire' ...
 'michael j. de la merc and rachel abram macy’ is said to receiv takeov approach by hudson’ bay - the new york time'
 'alex ansari nato, russia to hold parallel exercis in balkan'
 'david swanson what keep the f-35 aliv']


In [ ]:
print(y)

[1 0 1 ... 0 1 1]


In [ ]:
y.shape

(20800,)

In [ ]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [ ]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 273270 stored elements and shape (20800, 21840)>
  Coords	Values
  (0, 811)	0.3400125470717784
  (0, 3407)	0.3147612133526859
  (0, 3956)	0.21133365893512257
  (0, 4767)	0.30811924379137356
  (0, 4999)	0.23431383403253717
  (0, 5260)	0.2540066973529246
  (0, 6440)	0.1997517703057157
  (0, 8839)	0.1914679642820427
  (0, 9602)	0.15015156086333914
  (0, 9699)	0.2121959470951579
  (0, 10754)	0.25009974547778013
  (0, 11079)	0.3112890659869252
  (0, 16483)	0.22076398151078291
  (0, 19112)	0.2547006225991656
  (0, 19452)	0.2735442639240402
  (0, 20083)	0.18513603892347877
  (1, 2262)	0.28913616904285955
  (1, 2717)	0.15265747286920414
  (1, 3104)	0.3822119257689172
  (1, 3782)	0.18823022292597608
  (1, 4732)	0.2593840730087929
  (1, 7107)	0.7025382285074343
  (1, 8615)	0.1956590066021249
  (1, 13145)	0.15807218823479888
  (1, 20433)	0.2957534028592017
  :	:
  (20797, 14916)	0.23963213836343306
  (20797, 15152)	0.2636666682425273
 

Splitting the dataset into the Training set and Test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [ ]:
print(X_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 204968 stored elements and shape (15600, 21840)>
  Coords	Values
  (0, 2767)	0.3246631195818257
  (0, 3782)	0.1821534528762674
  (0, 5531)	0.3004837785903904
  (0, 6344)	0.42220525890880645
  (0, 8615)	0.1893424078498452
  (0, 9602)	0.19633532907978324
  (0, 9949)	0.41157578415983137
  (0, 13810)	0.42220525890880645
  (0, 18231)	0.2609893481955484
  (0, 18682)	0.23154248847307835
  (0, 20207)	0.22155996274218945
  (1, 0)	0.2743502189006602
  (1, 308)	0.3365282395340966
  (1, 2738)	0.3365282395340966
  (1, 6083)	0.24212247800120368
  (1, 8173)	0.3437281607700763
  (1, 8547)	0.2783342160000793
  (1, 8850)	0.21864799333951973
  (1, 9844)	0.21882660554665462
  (1, 13063)	0.12928757047340525
  (1, 14182)	0.30041483217205806
  (1, 17718)	0.40847786035707045
  (1, 18563)	0.2729040651380624
  (2, 93)	0.33825742620536137
  (2, 1429)	0.2577921679392103
  :	:
  (15598, 7823)	0.30903505058393216
  (15598, 9175)	0.1133536154909605
  (155

In [ ]:
print(y_train)

[1 1 0 ... 0 0 0]


In [ ]:
print(X_test)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 68302 stored elements and shape (5200, 21840)>
  Coords	Values
  (0, 2482)	0.2782506650218778
  (0, 5381)	0.33501996189039557
  (0, 5815)	0.31822137970012243
  (0, 9857)	0.31822137970012243
  (0, 12694)	0.09602373685949515
  (0, 12797)	0.37685051861993396
  (0, 13579)	0.3823627488929833
  (0, 14181)	0.3823627488929833
  (0, 14545)	0.36963628747688665
  (0, 18488)	0.08198413064645962
  (0, 18642)	0.10118704500547811
  (0, 20655)	0.10284883991702437
  (1, 1638)	0.2852616502090759
  (1, 5381)	0.3575263480148212
  (1, 5877)	0.35318066526549613
  (1, 9175)	0.1364102973359248
  (1, 10340)	0.3229079124778032
  (1, 12694)	0.10247453843763239
  (1, 12911)	0.31221937682342854
  (1, 13338)	0.35867453295592694
  (1, 17335)	0.31221937682342854
  (1, 19261)	0.3061843387054754
  (1, 20091)	0.3395992498549806
  (2, 899)	0.49026786547710655
  (2, 1638)	0.3873517719111919
  :	:
  (5197, 5709)	0.29177433460005936
  (5197, 6802)	0.2829959502378

In [ ]:
print(y_test)

[0 1 1 ... 0 1 0]


Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Training the Logistic Regression model on the Training set

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)

LogisticRegression(random_state=0)

In [ ]:
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [1 1]
 [1 1]
 ...
 [0 0]
 [1 1]
 [0 0]]


Making the Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[2520   44]
 [ 102 2534]]


0.9719230769230769

Making a Predictive System

In [ ]:
X_new = X_test[3]

prediction = classifier.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[0]
The news is Real
